In [265]:
import fitz
import regex as re
import pandas as pd

def read_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])
    return text

pdf_path = "calendario_universitario.pdf"
full_content = read_pdf(pdf_path)

In [266]:
def extract_from_text(text, start_point):
    pattern = rf"{start_point}(.*)"
    res = re.search(pattern, text, re.DOTALL)
    return res.group(0).strip() if res else "Texto não encontrado"

In [267]:
months_to_number = {
    "janeiro": 1,
    "fevereiro": 2,
    "março": 3,
    "abril": 4,
    "maio": 5,
    "junho": 6,
    "julho": 7,
    "agosto": 8,
    "setembro": 9,
    "outubro": 10,
    "novembro": 11,
    "dezembro": 12
}

number_to_month = {id: month for month, id in months_to_number.items()}

In [268]:
full_content

'A\nMINISTÉRIO DA EDUCAÇÃO\nUNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE\nRESOLUÇÃO No 022/2024-CONSAD, de 27 de junho de 2024.\nAprova, à unanimidade de votos, alteração do anexo da Resolução nº 059/2023-\nCONSAD, de 16 de novembro de 2023, que aprova o Calendário Universitário 2024\nda Universidade Federal do Rio Grande do Norte - UFRN, em razão da adesão de\ndocentes da UFRN à paralisação nacional no período de 22 de abril a 21 de junho\nde 2024.\nO VICE-REITOR DA UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE faz saber que o\nConselho de Administração - CONSAD, usando das atribuições que lhe confere o inciso IX, do artigo 19\ndo Estatuto da UFRN,\nCONSIDERANDO a Resolução nº 059/2023-CONSAD, de 16 de novembro de 2023, publicada no\nBoletim de Serviço no 217/2023, de 20 de novembro de 2023;\nCONSIDERANDO a Resolução nº 025/2024-CONSEPE, de 27 de junho de 2024;\nCONSIDERANDO o Ofício no 1/2024-ASSPROGRAD, de 25 de junho de 2024;\nCONSIDERANDO o que consta no processo no 23077.151596/2023

In [269]:
content = extract_from_text(full_content, "JANEIRO/2024")

In [270]:
content = content.split('\n')

In [271]:
def extract_tables(content:list[str]) -> pd.DataFrame:
    cleaned_content = [c for c in content if c not in ['', 'A']]
    
    re_month_year = r'^\b(?:JAN|FEV|MARÇ|ABR|MAI|JUN|JUL|AGO|SET|OUT|NOV|DEZ)[A-Z]*/\d{4}\b$'
    re_bad_month_year = r'^\b(?:JAN|FEV|MARÇ|ABR|MAI|JUN|JUL|AGO|SET|OUT|NOV|DEZ)[A-Z].*/(?:JAN|FEV|MARÇ|ABR|MAI|JUN|JUL|AGO|SET|OUT|NOV|DEZ)[A-Z].*'
    re_just_day = r'^\d{2}\b$'
    re_init_end_days = r'^\b\d{2} [a,e] \d{2}\b$'
    re_init_end_days_diff_months = r'^\b\d{2} [a,e] \d{2}\b\/\d{2}\b$'



    month, year = None, None
    initial_day, terminal_day = None, None
    
    text = []
    days_init = []
    days_term = []
    months_init = []
    months_term = []
    years = []
    
    for word in cleaned_content:
        matches_month = re.findall(re_month_year, word)
        matches_bad_month = re.findall(re_bad_month_year, word)
        matches_just_day = re.findall(re_just_day, word)
        matches_inital_terminal_days = re.findall(re_init_end_days, word)
        matches_inital_terminal_days_diff_months = re.findall(re_init_end_days_diff_months, word)
        
        if matches_month:
            month, year = matches_month[0].split('/')
            month = month.capitalize()
        
        elif matches_bad_month:
            month, year = None, None
        
        else:
            if month and year:
                if matches_just_day:
                    initial_day = matches_just_day[0]
                    terminal_day = matches_just_day[0]
                    text.append([])
                    days_init.append(initial_day)
                    days_term.append(terminal_day)
                    months_init.append(month)
                    months_term.append(month)
                    years.append(year)

                elif matches_inital_terminal_days:
                    matche = matches_inital_terminal_days[0]
                    dates = matche.split('a') if 'a' in matche else matche.split('e')
                    initial_day, terminal_day  = dates[0].strip(), dates[1].strip()
                    text.append([])
                    days_init.append(initial_day)
                    days_term.append(terminal_day)
                    months_init.append(month)
                    months_term.append(month)
                    years.append(year)

                elif matches_inital_terminal_days_diff_months:
                    matche = matches_inital_terminal_days_diff_months[0]
                    dates = matche.split('a') if 'a' in matche else matche.split('e')
                    initial_day, terminal_day  = dates[0].strip(), dates[1].strip()
                    text.append([])
                    days_init.append(initial_day)
                    days_term.append(terminal_day.split('/')[0])
                    months_init.append(month)
                    months_term.append(
                        number_to_month[int(terminal_day.split('/')[-1])].capitalize()
                    )
                    years.append(year)
                else:
                    text[-1].append(word)

    data =  dict(
        text = text,
        days_init = days_init,
        days_term = days_term,
        months_init = months_init,
        months_term = months_term,
        years = years,
    )

    return pd.DataFrame(data)
                
tables_df = extract_tables(content)

In [272]:
tables_df.text = tables_df.text.apply(lambda texts: ' '.join(texts))

In [273]:
tables_df

,text,days_init,days_term,months_init,months_term,years
0,Confraternização Universal – Feriado Nacional.,01,01,Janeiro,Janeiro,2024
1,"Início da vigência, no SIGAA, do período letiv...",03,03,Janeiro,Janeiro,2024
2,Recálculo dos alunos para a realização da matr...,04,04,Janeiro,Janeiro,2024
3,"Último dia para envio à PROGRAD, pelas coorden...",08,08,Janeiro,Janeiro,2024
4,"Período para solicitação, via SIGAA, de suspen...",04,08,Janeiro,Janeiro,2024
...,...,...,...,...,...,...
144,Desligamento por decurso de prazo máximo no pe...,24,24,Fevereiro,Fevereiro,2025
145,Processamento da matrícula dos alunos regulare...,26,28,Fevereiro,Fevereiro,2025
146,Matrícula automática dos ingressantes 2025.1.,06,07,Março,Março,2025
147,Semana de Avaliação e Planejamento pelos Depar...,10,14,Março,Março,2025


In [277]:
tables_df.to_csv('calendar_data.csv', index=False)

In [274]:
def reconstruct_text(df: pd.DataFrame) -> str:
    return df.apply(lambda r: f'De {r.days_init} de {r.months_init} de {r.years} - {r.text}', axis=1).to_list()


good_texts = reconstruct_text(tables_df)
good_text = '\n'.join(good_texts)

In [275]:
print(good_text)

De 01 de Janeiro de 2024 até 01 de Janeiro de 2024 - Confraternização Universal – Feriado Nacional.
De 03 de Janeiro de 2024 até 03 de Janeiro de 2024 - Início da vigência, no SIGAA, do período letivo 2024.1.
De 04 de Janeiro de 2024 até 04 de Janeiro de 2024 - Recálculo dos alunos para a realização da matrícula 2024.1 e geração das listas de alunos aptos a colar grau 2023.2, pela STI.
De 08 de Janeiro de 2024 até 08 de Janeiro de 2024 - Último dia para envio à PROGRAD, pelas coordenações de curso, dos pedidos de permuta de turno, cadastro de ênfase e de mudança de estrutura curricular, com vigência a partir de 2024.1.
De 04 de Janeiro de 2024 até 08 de Janeiro de 2024 - Período para solicitação, via SIGAA, de suspensão de programa a posteriori referente ao período letivo 2023.2.
De 05 de Janeiro de 2024 até 05 de Janeiro de 2024 - Matrícula extraordinária para o período letivo especial de férias de verão.
De 06 de Janeiro de 2024 até 06 de Janeiro de 2024 - Dia de Santos Reis – Feriad

In [276]:
with open("cleaned_calendar.txt", "w") as text_file:
    text_file.write(good_text)